# 🎨 Color+Texture Mosaic — Performance & Scaling Report (Tile Size = 16 px)

This notebook measures runtime and quality metrics of the mosaic system for different **grid sizes** and compares **vectorized** vs **loop-based** matching.

**What you'll get:**
1. A timing table for grid sizes: **16×16, 32×32, 64×64, 128×128** (vectorized pipeline)
2. A metrics table (MSE, PSNR, SSIM variants) across the same grid sizes
3. A comparison table of execution time: **vectorized vs loop-based** (for 16×16, 32×32, 64×64)

**Assumptions:**
- Tile size is fixed at **16 px** (uses a prebuilt cache NPZ in `./cache/`)
- You have at least one test image available (e.g., `./samples/your_image.jpg`)
- Cache files follow pattern: `cache/tiles_s16_allrotflips_*.npz`


## 0) Setup (Colab-friendly)
If running in Colab:

- Upload your **`cache/`** folder that contains the `tiles_s16_allrotflips_*.npz` file(s)
- Upload a sample image under `samples/` (or change `IMG_PATH` below)

You can use the cells below to create folders and upload files.

In [ ]:
# (Optional) Colab helpers: create dirs and upload files
import os
os.makedirs('cache', exist_ok=True)
os.makedirs('samples', exist_ok=True)

try:
    from google.colab import files  # type: ignore
    print("Colab detected. You can upload files with files.upload().")
    # Example: files.upload()  # Uncomment to open file picker
except Exception:
    print("Not running in Colab (or google.colab not available). Proceeding...")

## 1) Imports & Core Helpers (self-contained)
This cell defines the minimal functions needed (taken from the project) so the notebook can run standalone with just the cache + an input image.

In [ ]:
import glob, json, time, math
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass

from skimage.color import rgb2lab
from skimage.filters import sobel_h, sobel_v
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

CACHE_DIR = "cache"

def load_cache_by_size(tile_size: int):
    pattern = os.path.join(CACHE_DIR, f"tiles_s{tile_size:02d}_allrotflips_*.npz")
    matches = sorted(glob.glob(pattern))
    if not matches:
        raise FileNotFoundError(f"No cache for tile size {tile_size}: expected {pattern}")
    path = matches[-1]
    z = np.load(path, allow_pickle=False)
    key = json.loads(bytes(z["key_json"]).decode())
    cache = {
        "key": key,
        "tiles": z["tiles"].astype(np.uint8),
        "tile_means_lab_rot": z["tile_means_lab_rot"].astype(np.float32),
        "tile_texture_rot": z["tile_texture_rot"].astype(np.float32),
        "transform_ids": z["transform_ids"].astype(np.int16),
        "file_index": z["file_index"].astype(np.int32),
        "tile_size": int(z["tile_size"]),
    }
    return cache

def resize_and_crop_to_grid(img: np.ndarray, cell: int, grid: tuple[int, int]) -> np.ndarray:
    gy, gx = grid
    target_h, target_w = gy * cell, gx * cell
    pil = Image.fromarray(img)
    scale = max(target_w / pil.width, target_h / pil.height)
    new_w, new_h = int(np.ceil(pil.width * scale)), int(np.ceil(pil.height * scale))
    pil = pil.resize((new_w, new_h), Image.Resampling.LANCZOS)
    left = (new_w - target_w) // 2
    top = (new_h - target_h) // 2
    pil = pil.crop((left, top, left + target_w, top + target_h))
    return np.asarray(pil, dtype=np.uint8)

def quantize_colors_pillow(img: np.ndarray, k: int = 16) -> np.ndarray:
    if k <= 0:
        return img
    q = Image.fromarray(img).quantize(colors=k, method=Image.Quantize.FASTOCTREE).convert("RGB")
    return np.asarray(q, dtype=np.uint8)

def cell_lab_means(img: np.ndarray, cell: int) -> np.ndarray:
    H, W = img.shape[:2]
    gy, gx = H // cell, W // cell
    lab = rgb2lab(img / 255.0)
    blocks = lab.reshape(gy, cell, gx, cell, 3).swapaxes(1, 2)
    mu = blocks.mean(axis=(2, 3))
    return mu.astype(np.float32)

def cell_texture_scalar(img: np.ndarray, cell: int) -> np.ndarray:
    H, W = img.shape[:2]
    gy, gx = H // cell, W // cell
    L = rgb2lab(img / 255.0)[..., 0].astype(np.float32)
    gy_ = sobel_h(L)
    gx_ = sobel_v(L)
    mag = np.hypot(gx_, gy_)
    blocks = mag.reshape(gy, cell, gx, cell).swapaxes(1, 2)
    t = blocks.mean(axis=(2, 3))
    return t.astype(np.float32)

def reconstruct_mosaic(tile_idx: np.ndarray, tiles: np.ndarray) -> np.ndarray:
    gy, gx = tile_idx.shape
    s = tiles.shape[1]
    out = np.zeros((gy * s, gx * s, 3), dtype=np.uint8)
    for iy in range(gy):
        for ix in range(gx):
            out[iy*s:(iy+1)*s, ix*s:(ix+1)*s] = tiles[tile_idx[iy, ix]]
    return out

def mse(a: np.ndarray, b: np.ndarray) -> float:
    a = a.astype(np.float32)
    b = b.astype(np.float32)
    return float(np.mean((a - b) ** 2))

def psnr_rgb(a: np.ndarray, b: np.ndarray) -> float:
    return float(psnr(a, b, data_range=255))

def ssim_rgb(a: np.ndarray, b: np.ndarray) -> float:
    return float(ssim(a, b, channel_axis=2, data_range=255))

def ssim_blockwise(proc: np.ndarray, mosaic: np.ndarray, cell: int) -> float:
    H, W = proc.shape[:2]
    gy, gx = H // cell, W // cell
    scores = []
    k = min(cell, 11)
    if k % 2 == 0:
        k -= 1
    k = max(3, k)
    for iy in range(gy):
        for ix in range(gx):
            a = proc[iy*cell:(iy+1)*cell, ix*cell:(ix+1)*cell]
            b = mosaic[iy*cell:(iy+1)*cell, ix*cell:(ix+1)*cell]
            try:
                s = ssim(a, b, channel_axis=2, data_range=255, win_size=k)
            except Exception:
                ag = a.mean(axis=2)
                bg = b.mean(axis=2)
                kk = min(k, min(ag.shape))
                if kk % 2 == 0:
                    kk -= 1
                kk = max(3, kk)
                s = ssim(ag, bg, data_range=255, win_size=kk)
            scores.append(float(s))
    return float(np.mean(scores)) if scores else 0.0

def ssim_per_channel(a: np.ndarray, b: np.ndarray) -> tuple[float, float, float, float]:
    rs = float(ssim(a[...,0], b[...,0], data_range=255))
    gs = float(ssim(a[...,1], b[...,1], data_range=255))
    bs = float(ssim(a[...,2], b[...,2], data_range=255))
    return rs, gs, bs, float((rs + gs + bs) / 3.0)

def ssim_global_gray(a: np.ndarray, b: np.ndarray) -> float:
    ag = a.mean(axis=2)
    bg = b.mean(axis=2)
    win = min(ag.shape)
    if win % 2 == 0:
        win -= 1
    win = max(3, win)
    return float(ssim(ag, bg, data_range=255, win_size=win))

def ssim_multiscale_gray(a: np.ndarray, b: np.ndarray) -> float:
    def to_gray(x):
        return x.mean(axis=2).astype(np.float32)
    ag, bg = to_gray(a), to_gray(b)
    scores = []
    for scale in (1.0, 0.5, 0.25):
        if scale == 1.0:
            Ag, Bg = ag, bg
        else:
            new_h = max(2, int(round(ag.shape[0] * scale)))
            new_w = max(2, int(round(ag.shape[1] * scale)))
            Ag = np.array(Image.fromarray(ag).resize((new_w, new_h), Image.Resampling.BICUBIC))
            Bg = np.array(Image.fromarray(bg).resize((new_w, new_h), Image.Resampling.BICUBIC))
        win = min(Ag.shape)
        if win % 2 == 0:
            win -= 1
        win = max(3, win)
        scores.append(float(ssim(Ag, Bg, data_range=255, win_size=win)))
    return float(np.mean(scores)) if scores else 0.0

def texture_aware_mapping(block_avgs, block_textures, tile_means, tile_textures, alpha=0.7):
    gy, gx, _ = block_avgs.shape
    B_colors = block_avgs.reshape(-1, 3)
    B_textures = block_textures.reshape(-1)
    color_dists = ((B_colors[:, None, :] - tile_means[None, :, :]) ** 2).sum(axis=2)
    texture_dists = (B_textures[:, None] - tile_textures[None, :]) ** 2
    color_dists = color_dists / (color_dists.max() + 1e-8)
    texture_dists = texture_dists / (texture_dists.max() + 1e-8)
    combined = alpha * color_dists + (1 - alpha) * texture_dists
    idx = combined.argmin(axis=1).reshape(gy, gx)
    return idx.astype(np.int32)

def match_color_texture_naive(mu_lab, tex, tile_means, tile_textures, alpha=0.7):
    """Naive Python-loop version (for timing comparison)."""
    gy, gx, _ = mu_lab.shape
    Bc = mu_lab.reshape(-1, 3)
    Bt = tex.reshape(-1)
    T = tile_means
    U = tile_textures
    B = Bc.shape[0]
    Tn = T.shape[0]
    # compute maxima for normalization
    color_raw_max = 0.0
    text_raw_max = 0.0
    for i in range(B):
        for j in range(Tn):
            cd = float(np.sum((Bc[i] - T[j])**2))
            td = float((Bt[i] - U[j])**2)
            if cd > color_raw_max: color_raw_max = cd
            if td > text_raw_max: text_raw_max = td
    color_norm = max(color_raw_max, 1e-8)
    text_norm = max(text_raw_max, 1e-8)
    idx_flat = np.empty(B, dtype=np.int32)
    for i in range(B):
        best_j, best_d = -1, float("inf")
        for j in range(Tn):
            cd = float(np.sum((Bc[i] - T[j])**2)) / color_norm
            td = float((Bt[i] - U[j])**2) / text_norm
            d = alpha * cd + (1 - alpha) * td
            if d < best_d:
                best_d = d
                best_j = j
        idx_flat[i] = best_j
    return idx_flat.reshape(gy, gx)

@dataclass
class Timing:
    preprocess: float
    features: float
    match: float
    reconstruct: float
    total: float

def run_once(image_rgb, grid, tile_size, quant_k, alpha, tile_means, tile_textures, tiles, vectorized=True, do_metrics=True):
    rows, cols = grid
    t0 = time.perf_counter()
    # preprocess
    t1 = time.perf_counter()
    proc = resize_and_crop_to_grid(image_rgb, cell=tile_size, grid=(rows, cols))
    if quant_k > 0:
        proc = quantize_colors_pillow(proc, k=quant_k)
    t_pre = time.perf_counter() - t1
    # features
    t2 = time.perf_counter()
    mu_lab = cell_lab_means(proc, tile_size)
    tex = cell_texture_scalar(proc, tile_size)
    t_feat = time.perf_counter() - t2
    # matching
    t3 = time.perf_counter()
    if vectorized:
        idx = texture_aware_mapping(mu_lab, tex, tile_means, tile_textures, alpha)
    else:
        idx = match_color_texture_naive(mu_lab, tex, tile_means, tile_textures, alpha)
    t_match = time.perf_counter() - t3
    # reconstruct
    t4 = time.perf_counter()
    mosaic = reconstruct_mosaic(idx, tiles)
    t_reco = time.perf_counter() - t4
    total = time.perf_counter() - t0

    metrics = {}
    if do_metrics:
        metrics = {
            "MSE": mse(proc, mosaic),
            "PSNR": psnr_rgb(proc, mosaic),
            "SSIM_pixel": ssim_rgb(proc, mosaic),
            "SSIM_block": ssim_blockwise(proc, mosaic, tile_size),
            "SSIM_global_gray": ssim_global_gray(proc, mosaic),
            "SSIM_multiscale_gray": ssim_multiscale_gray(proc, mosaic),
        }
    return mosaic, Timing(t_pre, t_feat, t_match, t_reco, total), proc, metrics


## 2) Experiment Config
- Tile size fixed to **16 px**
- Grids: 16×16, 32×32, 64×64, 128×128
- Repeat runs for stability (adjust `REPEATS` if needed)

In [ ]:
TILE_SIZE = 16
ALPHA = 0.7
GRIDS = [(16,16),(32,32),(64,64),(128,128)]
REPEATS = 3  # increase to 5 for more stable averages
QUANT_K = 0  # keep off for timing fairness

# Choose one sample image
IMG_PATH = "samples/beach.jpg"  # change path as needed
img_rgb = np.asarray(Image.open(IMG_PATH).convert("RGB"), dtype=np.uint8)

cache = load_cache_by_size(TILE_SIZE)
tiles = cache["tiles"]
tile_means = cache["tile_means_lab_rot"]
tile_textures = cache["tile_texture_rot"]
print("Loaded cache with tiles:", tiles.shape)

## 3) Timing across grid sizes (vectorized pipeline)
This produces a table with timings per stage and total time for each grid size.

In [ ]:
rows = []
for (gy,gx) in GRIDS:
    totals = []
    pre_list, feat_list, match_list, reco_list, total_list = [], [], [], [], []
    for _ in range(REPEATS):
        _, t, _, _ = run_once(img_rgb, (gy,gx), TILE_SIZE, QUANT_K, ALPHA,
                              tile_means, tile_textures, tiles, vectorized=True, do_metrics=False)
        pre_list.append(t.preprocess)
        feat_list.append(t.features)
        match_list.append(t.match)
        reco_list.append(t.reconstruct)
        total_list.append(t.total)
    rows.append({
        "Grid": f"{gy}x{gx}",
        "Blocks(B)": gy*gx,
        "Preprocess(s)": np.mean(pre_list),
        "Features(s)": np.mean(feat_list),
        "Matching(s)": np.mean(match_list),
        "Reconstruct(s)": np.mean(reco_list),
        "Total(s)": np.mean(total_list),
    })
timing_table = pd.DataFrame(rows)
display(timing_table)


## 4) Quality metrics across grid sizes (vectorized pipeline)
Computes MSE, PSNR, and SSIM variants for each grid size.

In [ ]:
metrics_rows = []
for (gy,gx) in GRIDS:
    # single run per grid for metrics (can average if you prefer)
    mosaic, t, proc, mets = run_once(img_rgb, (gy,gx), TILE_SIZE, QUANT_K, ALPHA,
                                     tile_means, tile_textures, tiles, vectorized=True, do_metrics=True)
    metrics_rows.append({
        "Grid": f"{gy}x{gx}",
        "Blocks(B)": gy*gx,
        "MSE": mets["MSE"],
        "PSNR(dB)": mets["PSNR"],
        "SSIM_pixel": mets["SSIM_pixel"],
        "SSIM_block": mets["SSIM_block"],
        "SSIM_global_gray": mets["SSIM_global_gray"],
        "SSIM_multiscale_gray": mets["SSIM_multiscale_gray"],
    })
metrics_table = pd.DataFrame(metrics_rows)
display(metrics_table)


## 5) Vectorized vs Loop-based comparison (16×16, 32×32, 64×64)
We compare matching time and total time. (Loop-based can be slow for large grids; we omit 128×128.)

In [ ]:
GRIDS_COMPARE = [(16,16),(32,32),(64,64)]
rows_cmp = []
for (gy,gx) in GRIDS_COMPARE:
    # vectorized
    _, t_vec, _, _ = run_once(img_rgb, (gy,gx), TILE_SIZE, QUANT_K, ALPHA,
                              tile_means, tile_textures, tiles, vectorized=True, do_metrics=False)
    # loop-based
    _, t_loop, _, _ = run_once(img_rgb, (gy,gx), TILE_SIZE, QUANT_K, ALPHA,
                               tile_means, tile_textures, tiles, vectorized=False, do_metrics=False)
    rows_cmp.append({
        "Grid": f"{gy}x{gx}",
        "Blocks(B)": gy*gx,
        "Matching Vec (s)": t_vec.match,
        "Matching Loop (s)": t_loop.match,
        "Total Vec (s)": t_vec.total,
        "Total Loop (s)": t_loop.total,
        "Speedup (match)": t_loop.match / max(t_vec.match, 1e-9),
        "Speedup (total)": t_loop.total / max(t_vec.total, 1e-9),
    })
compare_table = pd.DataFrame(rows_cmp)
display(compare_table)


## 6) Scaling analysis (brief)
We estimate how matching time scales with the number of blocks `B = rows×cols` using a log–log linear fit.

In [ ]:
def log_fit(x, y):
    lx = np.log(np.asarray(x, dtype=np.float64))
    ly = np.log(np.asarray(y, dtype=np.float64))
    k, b = np.polyfit(lx, ly, deg=1)  # y ≈ e^b * x^k
    return float(k), float(math.exp(b))

# use vectorized matching times from timing_table
Bs = timing_table["Blocks(B)"].tolist()
vec_match_times = timing_table["Matching(s)"].tolist()
k_vec, c_vec = log_fit(Bs, vec_match_times)

analysis_text = (
    f"Vectorized matching scales approximately as T(B) ≈ {c_vec:.3g} * B^{k_vec:.2f}.\n"
    f"(Here B is the number of grid cells; tile count is fixed by the cache.)"
)
print(analysis_text)

## 7) Optional: Quick plots (if you want visuals)
These are optional to include in the report.

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(timing_table["Blocks(B)"], timing_table["Total(s)"], marker='o', label='Total (vectorized)')
plt.plot(timing_table["Blocks(B)"], timing_table["Matching(s)"], marker='o', label='Matching (vectorized)')
plt.xlabel('Blocks B = rows×cols')
plt.ylabel('Time (s)')
plt.title('Scaling with Grid Size (Tile=16)')
plt.grid(True)
plt.legend()
plt.show()